# Homework 2
**Christian Steinmetz**

Due on November 22nd

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

## Problem formulation
We set up an experimental framework to study various aspects of overfitting. 
The input space is $X = [−1, 1]$ with uniform input probability density, $P(x) =\frac{1}{2}$.
We consider the two models $\mathcal{H}_2$ and $\mathcal{H}_{10}$. 
The target function is a polynomial of degree $Q_f$, which we write as $f(x)=\sum_{q=0}^{Q_f} a_q L_q(x)$, where $Lq(x)$ are the Legendre polynomials.
We use the Legendre polynomials because they are a convenient orthogonal basis for the polynomials on $[−1, 1]$.
The first two Legendre polynomials are $L_0(x) = 1$, $L_1(x) = x$.
The higher order Legendre polynomials are defined by the recursion:

\begin{equation}
    L_k(x) = \frac{2k-1}{k} x L_{k-1}(x) - \frac{k-1}{k} L_{k-2}(x)
\end{equation}

The data set is $\mathcal{D} = (x_1, y_1), ..., (x_N , y_N)$, where $y_n = f (x_n)+\sigma \epsilon_n$ and $\epsilon_n$ are i.i.d. standard Normal random variables.

For a single experiment, with specified values for $Q_f$, $N$, $\sigma$, generate a random degree $Q_f$ target function by selecting coefficients $a_q$ independently from a standard Normal distribution, rescaling them so that $\mathbb{E}_{a,x}[f^2]=1$. Generate a data set, selecting
$x_1,...,x_N$ independently from $P(x)$ and $y_n = f (x_n)+\sigma \epsilon_n$. 

Let $g_2$ and $g_{10}$ be the best fit hypotheses to the data from $\mathcal{H_2}$ and $\mathcal{H}_{10}$, respectively, with respective out-of sample errors $E_{out}(g_2)$ and $E_{out}(g_{10})$.

In [2]:
def legendre_poly(k, x):
    if k == 0:
        return 1
    elif k == 1:
        return x
    else:
        return ((2*k-1)/k) * x * legendre_poly(k-1, x) - ((k-1)/k) * legendre_poly(k-2, x)

In [407]:
def generate_a_q(Q_f=None):
    
    if not Q_f:
        # deteremine the degree, Q_f
        Q_f = np.random.randint(3, 10)
    
    # sample values for a_q
    a_q = np.random.standard_normal(size=Q_f)
    
    #a_q[0] = 0 
    # for simplicity we hold the first coef at 0
    # which means that E(f(x)) = 0
    
    # normalize by finding the variance of f(x)
    Ef2 = np.array([a**2/((2*q)+1) for q, a in enumerate(a_q)]).sum() #+ a_q[0]**2
    
    return a_q / np.sqrt(Ef2)

In [408]:
def f(x, a_q):
    #return np.sum([a * legendre_poly(k, x) for k, a in enumerate(a_q)])
    
    ploy = legendre(len(a_q))

In [409]:
def f(x, a_q):
    return np.polynomial.legendre.legval(x, a_q)

In [410]:
def generate_dataset(f, a_q, sigma, N):
    X = (np.random.rand(N) * 2) - 1 # uniform over [-1, 1]
    Y = np.polynomial.legendre.legval(X, a_q) + (sigma * np.random.standard_normal(size=N))

    return X, Y

In [411]:
a_q = generate_a_q(Q_f=2)

X, Y = generate_dataset(f, a_q, 0.0, 1000000)
print(f"X    mean: {X.mean():.2f}, var: {X.var():.2f}")
print(f"Y    mean: {Y.mean():.2f}, var: {Y.var():.2f}")

X    mean: 0.00, var: 0.33
Y    mean: 0.84, var: 0.29


**(a) Why do we normalize $f$?** [Hint: how would you interpret σ?]

Based upon it's appearence in $y_n$, $\sigma$ appears to be a scaling factor of the Gaussian noise term $\sigma \epsilon_n$, since $\epsilon_n \sim N(0, 1)$ . This scaling term has the effect of increasing the variance of the distribution of this noise term, since larger values of $\sigma$ will result in greater spread in likely values of the noise. 

We know also that our polynomial coefficents, $a_q$, are drawn from a standard normal as well, and we also know that the second moment of a random variable $X$ is given by $\mathbb{E}(X^2)$, which corresponds to its varience. By normalizing our coefficients, $a_q$, of the target function $f(x)$, so that $\mathbb{E}_{x}[f^2]=1$, we enforce the output of our function to be a normal random variable ($\mu=a_0$ and $\sigma=\text{Var}(f)$).

From a higher level, we can see that $y_n$ is the sum of two standard normal random variables, where the term $\sigma$ controls the relative influence of the gaussian noise term, $\sigma\epsilon_n$. When $\sigma=1$ the output of $y_n$ is the direct sum of two standard normals, whereas when $\sigma=0$ the output is just the output of $f$.

In conclusion, we propose that the input, $X$, to our target function, $f$, can be modled as a uniform random variable with $P(x) = \frac{1}{2}$. The output of is given as $y_n = f (x_n)+\sigma \epsilon_n$, and we notice is is the sum of our normalized function $f \sim N(0,1)$ and the noise term $\sigma \epsilon_n \sim N(0,\sigma)$.

**(b) How can we obtain $g_2$, $g_{10}$?** [Hint: pose the problem as linear regression]

To find the best fit hypothesis we can use all of the datapoints in $D$ to fit a linear regression model within $\mathcal{H}_2$ and $\mathcal{H}_{10}$, where we attempt to minimize the in-sample error for each model. 


In [412]:
H_2 = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
X_2 = H_2.fit_transform(X.reshape(-1,1))

H_10 = preprocessing.PolynomialFeatures(degree=10, include_bias=False)
X_10 = H_10.fit_transform(X.reshape(-1,1))

In [413]:
g_2 = LinearRegression(fit_intercept=True)
g_2.fit(X_2, Y)

g_10 = LinearRegression()
g_10.fit(X_10, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**(c) How can we compute $E_{out}$ analytically for a given $g_{10}$?**

Since we know the target function, $f$, and our best fit hypothesis will also provide a function, we can measure the distance between these to calculate $E_{out}$. A perfect prediction of $f$ will mean that our best fit hypothesis matches the exact same target function exactly. 

More generally, for any predicted target function $f_n$, we can define the expected error as 

\begin{equation}
\mathbb{E}_{err} = \int_{X \times Y} \mathcal{L}(f_n(x), y) \rho(x,y) dxdy
\end{equation}

where $\mathcal{L}$ is a loss function (ex: $\mathcal{L}(\hat{y}, y) = \mathbb{E}[ (y - \hat{y})^2 ]$)   and $\rho(x,y)$ is the joint probabaility of $x$ and $y$. 

**(d) Vary $Q_f$, $N$, $\sigma$ and for each combination of parameters, run a large number of experiments, each time computing $E_{out}(g_2)$ and $E_{out}(g_{10})$.**

Averaging these out-of-sample errors gives estimates of the expected out-of-sample error for the given learning scenario ($Q_f$, $N$, $\sigma$) using $\mathcal{H_2}$ and $\mathcal{H_{10}}$. 

Let 

\begin{equation}
    E_{out}(\mathcal{H}_2) = \text{average over experiments}(E_{out}(g_2)), \\
    E_{out}(\mathcal{H}_{10}) = \text{average over experiments}(E_{out}(g_{10})).
\end{equation}

Define the overfit measure $E_{out}(\mathcal{H}_{10}) − E_{out}(\mathcal{H}_2)$. When is the overfit measure significantly positive (i.e. overfitting is serious) as opposed to significantly
negative? Try the choices $Q_f \in \{1, 2,..., 100\}, N \in \{20, 25,..., 120\}, \sigma^2 \in \{0, 0.05, 0.1,..., 2\}$.

Explain your observations.

In [427]:
def fit_models(Q_f, N, s):
    a_q = generate_a_q(Q_f=Q_f)
    X, Y = generate_dataset(f, a_q, s, N)
    
    H2 = preprocessing.PolynomialFeatures(degree=2, include_bias=True)
    X2 = H2.fit_transform(X.reshape(-1,1))
    g2 = LinearRegression(fit_intercept=True)
    g2.fit(X2, Y)
    
    H10 = preprocessing.PolynomialFeatures(degree=10, include_bias=True)
    X10 = H10.fit_transform(X.reshape(-1,1))
    g10 = LinearRegression()
    g10.fit(X10, Y)

    # this is an approx. of E_out
    x = np.linspace(-1,1,1000)
    y = np.polynomial.legendre.legval(x, a_q)
    y2_hat = g2.predict(H2.fit_transform(x.reshape(-1,1)))
    y10_hat = g10.predict(H10.fit_transform(x.reshape(-1,1)))
    
    print(a_q[0])
    print(y2_hat.mean())
    print(y10_hat.mean())
    print()
    print(a_q[0]**2)
    print(np.mean(y2_hat**2))
    print(np.mean(y10_hat**2))
    
    
    
    E_out_g2  = np.power(y - y2_hat,2).mean()
    E_out_g10 = np.power(y - y10_hat,2).mean()
    
    return E_out_g2, E_out_g10
    

In [429]:
fit_models(3, 1000000, 0.0)

0.7336239731172762
0.732268912905997
0.7322689129059974

0.538204133932378
0.9996722489517377
0.9996722489517369


(7.923258534876786e-29, 7.43938604675619e-29)

In [147]:
Q_f = np.array([1, 2, 3, 4, 5, 10, 20, 50])
N   = np.arange(20,121, 20)
Sig = [0.0, 1.0]#np.arange(0,2.05,0.05)
T   = np.arange(10)

n_iters = len(Q_f) * len(N) * len(Sig) * len(T)
iters = 0

logs = []

for q_f in Q_f:
    for n in N:
        for s in Sig:
            Eh2, Eh10 = [], []
            for t in T:
                Eg2, Eg10 = fit_models(q_f, n, s)
                Eh2.append(Eg2)
                Eh10.append(Eg10)
                iters += 1
                
                # print some progress
                sys.stdout.write(f"{iters:4d}/{n_iters:4d} - {(iters/n_iters)*100:2.2f}%\r")
                sys.stdout.flush()
            
            Eh2 = np.array(Eh2).mean()
            Eh10 = np.array(Eh10).mean()
            
            results = {
                "Q_f" : q_f,
                "N" : n,
                "Sigma" : s,
                "E_out(H2)" : Eh2,
                "E_out(H10)" : Eh10,
                "overfit" : Eh10 - Eh2}
            
            logs.append(results)
            
for entry in logs:
    print(entry)

{'Q_f': 1, 'N': 20, 'Sigma': 0.0, 'E_out(H2)': 0.0, 'E_out(H10)': 0.0, 'overfit': 0.0}
{'Q_f': 1, 'N': 20, 'Sigma': 1.0, 'E_out(H2)': 0.10264516525441272, 'E_out(H10)': 858.1534219632698, 'overfit': 858.0507767980154}
{'Q_f': 1, 'N': 40, 'Sigma': 0.0, 'E_out(H2)': 0.0, 'E_out(H10)': 0.0, 'overfit': 0.0}
{'Q_f': 1, 'N': 40, 'Sigma': 1.0, 'E_out(H2)': 0.04968258687305303, 'E_out(H10)': 1.09889383733648, 'overfit': 1.049211250463427}
{'Q_f': 1, 'N': 60, 'Sigma': 0.0, 'E_out(H2)': 0.0, 'E_out(H10)': 0.0, 'overfit': 0.0}
{'Q_f': 1, 'N': 60, 'Sigma': 1.0, 'E_out(H2)': 0.03688631796707949, 'E_out(H10)': 0.3901382022321957, 'overfit': 0.3532518842651162}
{'Q_f': 1, 'N': 80, 'Sigma': 0.0, 'E_out(H2)': 0.0, 'E_out(H10)': 0.0, 'overfit': 0.0}
{'Q_f': 1, 'N': 80, 'Sigma': 1.0, 'E_out(H2)': 0.033450791077470864, 'E_out(H10)': 0.1739913777525378, 'overfit': 0.14054058667506694}
{'Q_f': 1, 'N': 100, 'Sigma': 0.0, 'E_out(H2)': 0.0, 'E_out(H10)': 0.0, 'overfit': 0.0}
{'Q_f': 1, 'N': 100, 'Sigma': 1.0, 

**(e) Why do we take the average over many experiments?** Use the variance to select an acceptable number of experiments to average over.